In [14]:
#import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
#from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import r2_score, mean_squared_error
#import AllTogether as t
import seaborn as sns
#%matplotlib inline

<h3> Loading and exploring the Seattle Airbnb Data </h3>

In this first section, we are going to load and explore the Seattle Airbnb Data set. The aim is to identify the most important features which we can use to answer the business questions given above.

In [25]:
df_seattle_calendar = pd.read_csv('./seattle_airbnb_data/calendar.csv')
df_seattle_listings = pd.read_csv('./seattle_airbnb_data/listings.csv')
df_seattle_reviews = pd.read_csv('./seattle_airbnb_data/reviews.csv')

print(df_seattle_calendar.head())
print(df_seattle_listings.head())
print(df_seattle_reviews.head())

   listing_id        date available   price
0      241032  2016-01-04         t  $85.00
1      241032  2016-01-05         t  $85.00
2      241032  2016-01-06         f     NaN
3      241032  2016-01-07         f     NaN
4      241032  2016-01-08         f     NaN
        id                           listing_url       scrape_id last_scraped  \
0   241032   https://www.airbnb.com/rooms/241032  20160104002432   2016-01-04   
1   953595   https://www.airbnb.com/rooms/953595  20160104002432   2016-01-04   
2  3308979  https://www.airbnb.com/rooms/3308979  20160104002432   2016-01-04   
3  7421966  https://www.airbnb.com/rooms/7421966  20160104002432   2016-01-04   
4   278830   https://www.airbnb.com/rooms/278830  20160104002432   2016-01-04   

                                  name  \
0         Stylish Queen Anne Apartment   
1   Bright & Airy Queen Anne Apartment   
2  New Modern House-Amazing water view   
3                   Queen Anne Chateau   
4       Charming craftsman 3 bdm house 

In [23]:
df_boston = pd.read_csv('./boston_airbnb_data/calendar.csv')
df_boston.head()

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


In [22]:
df_boston.shape

(3585, 95)